In [1]:
import datetime
import dateutil

# `parser`：几乎能解析任何格式的日期字符串

这是 `dateutil` 最受欢迎的功能之一。通常使用 `datetime.strptime()` 解析字符串时，您必须精确指定日期的格式。而 `dateutil.parser.parse()` 可以智能地识别多种常见的日期格式，无需手动指定格式代码。

## 基础用法

In [ ]:
timestr_list = [
    "2023-10-27",
    "27 Oct 2023",
    "October 27, 2023, 10:30 AM",
    "2023/10/27 10:30:15",
    "2023 10 27 10:30:15",
    "2023-10-27 10:30:15",
]

# 无需指定格式，自动解析多种格式的字符串
for time_str in timestr_list:
    try:
        dt = dateutil.parser.parse(time_str)
        print(f"'{time_str}' -> {dt}")
    except ValueError as e:
        print(e)

'2023-10-27' -> 2023-10-27 00:00:00
'27 Oct 2023' -> 2023-10-27 00:00:00
'October 27, 2023, 10:30 AM' -> 2023-10-27 10:30:00
'2023/10/27 10:30:15' -> 2023-10-27 10:30:15
'2023 10 27 10:30:15' -> 2023-10-27 10:30:15
'2023-10-27 10:30:15' -> 2023-10-27 10:30:15


## 处理模糊的日期格式

`parser` 甚至可以处理像日/月/年顺序不明确的情况。例如，`01-02-03` 究竟是 2003年1月2日 还是 2001年2月3日？ 您可以通过参数来指定：

In [11]:
# 假设你知道日总是在前面
print(dateutil.parser.parse("01-02-03", dayfirst=True))
# 输出: 2003-02-01 00:00:00

# 假设你知道年总是在前面
print(dateutil.parser.parse("01-02-03", yearfirst=True))
# 输出: 2001-02-03 00:00:00

2003-02-01 00:00:00
2001-02-03 00:00:00


# `relativedelta`：强大的日期时间加减运算

`Python` 内置的 `datetime.timedelta` 只能处理天、秒、微秒等单位的计算，但无法直接处理像“一个月后”或“下一年”这样的相对时间。

因为每个月的天数不同，还存在闰年的问题。`relativedelta` 完美地解决了这个问题。

## 基础用法

In [15]:
today = datetime.date.today()
print(f"今天的日期: {today}")

今天的日期: 2025-10-11


In [17]:
# 计算未来和过去的日期
one_month_later = today + dateutil.relativedelta.relativedelta(months=1)
print(f"一个月后: {one_month_later}")

一个月后: 2025-11-11


In [ ]:
two_years_ago = today - dateutil.relativedelta.relativedelta(years=2)
print(f"两年前: {two_years_ago}")

In [18]:
# 复杂的日期计算
next_exam_date = today + dateutil.relativedelta.relativedelta(months=2, weeks=1, days=3)
print(f"2个月1周零3天后: {next_exam_date}")

2个月1周零3天后: 2025-12-21


## 计算两个日期之间的差值

`relativedelta` 也可以用来计算两个日期之间精确的年、月、日差。

In [20]:
birth_date = datetime.date(1990, 5, 15)
today = datetime.date(2023, 10, 27)

In [21]:
age = dateutil.relativedelta.relativedelta(today, birth_date)
print(f"年龄: {age.years} 年, {age.months} 个月, {age.days} 天")
# 输出: 年龄: 33 年, 5 个月, 12 天

年龄: 33 年, 5 个月, 12 天


## 获取特定日期

您还可以用它来找到例如“下周的星期三”或“这个月的最后一个星期五”。

In [22]:
today = datetime.date(2023, 10, 27) # 假设今天是周五
today

datetime.date(2023, 10, 27)

In [23]:
# 下一个周三
next_wednesday = today + dateutil.relativedelta.relativedelta(weekday=dateutil.relativedelta.WE(+1))
print(f"下一个周三是: {next_wednesday}") # WE(+1) 表示下一个周三

下一个周三是: 2023-11-01


In [24]:
# 这个月的最后一个周五
last_friday_of_month = today + dateutil.relativedelta.relativedelta(day=31, weekday=dateutil.relativedelta.FR(-1))
print(f"本月最后一个周五是: {last_friday_of_month}") # day=31 定位到月末, FR(-1) 表示从后往前数的第一个周五

本月最后一个周五是: 2023-10-27


# `rrule`：轻松处理重复性事件

如果您需要处理日历应用中的那种重复性规则（例如，每周二、周四开会，持续10次），`rrule` 会非常有用。它实现了 `iCalendar` 规范（RFC 5545）中的重复规则。

In [26]:
# 规则：从2023年11月1日开始，每周的周一和周五，总共发生10次
start_date = datetime.datetime(2023, 11, 1)
event_dates = list(dateutil.rrule.rrule(
    freq=dateutil.rrule.WEEKLY,              # 重复频率：每周
    count=10,                 # 总共次数
    byweekday=(dateutil.rrule.MO, dateutil.rrule.FR),       # 每周的周一和周五
    dtstart=start_date        # 开始日期
))

for dt in event_dates:
    print(dt.strftime("%Y-%m-%d (%a)")) # (%a) 显示星期的缩写

2023-11-03 (Fri)
2023-11-06 (Mon)
2023-11-10 (Fri)
2023-11-13 (Mon)
2023-11-17 (Fri)
2023-11-20 (Mon)
2023-11-24 (Fri)
2023-11-27 (Mon)
2023-12-01 (Fri)
2023-12-04 (Mon)


# `tz` 主要用于处理时区（Time Zone）信息

## 获取时区对象

在使用时区前，首先需要获取一个代表特定时区的对象。`dateutil.tz` 提供了几种便捷的方法。

最常用的方法是 `tz.gettz()`，它可以解析大部分标准的时区名称（IANA Time Zone database names，如 "America/New_York", "Asia/Shanghai"）。

In [27]:
# 获取特定时区的对象
tz_shanghai = dateutil.tz.gettz("Asia/Shanghai")
tz_new_york = dateutil.tz.gettz("America/New_York")
tz_singapore = dateutil.tz.gettz("Asia/Singapore") # 获取新加坡时区

# 获取UTC（协调世界时）时区
tz_utc = dateutil.tz.tzutc()

# 获取本地时区
tz_local = dateutil.tz.tzlocal()

print(f"上海时区: {tz_shanghai}")
print(f"纽约时区: {tz_new_york}")
print(f"新加坡时区: {tz_singapore}")
print(f"UTC时区: {tz_utc}")
print(f"本地时区: {tz_local}")

上海时区: tzfile('PRC')
纽约时区: tzfile('America/New_York')
新加坡时区: tzfile('Asia/Kuala_Lumpur')
UTC时区: tzutc()
本地时区: tzlocal()


## 创建一个带有时区的 datetime 对象

当你有一个 "naive" 的 `datetime` 对象时，你可以使用 `.replace()` 方法为其附加时区信息，使其变为 "aware" 对象。

场景：假设你知道一个时间 2025-10-11 14:00:00 是在北京时间（上海时区）发生的。

In [29]:
# 一个 "naive" 的时间对象
naive_dt = datetime.datetime(2025, 10, 11, 14, 0, 0)
print(f"无时区信息的datetime: {naive_dt}")

无时区信息的datetime: 2025-10-11 14:00:00


In [30]:
# 获取上海时区
shanghai_tz = dateutil.tz.gettz("Asia/Shanghai")
shanghai_tz

tzfile('PRC')

In [31]:
# 使用 replace() 方法附加时区信息
aware_dt = naive_dt.replace(tzinfo=shanghai_tz)
aware_dt

datetime.datetime(2025, 10, 11, 14, 0, tzinfo=tzfile('PRC'))

In [32]:
print(f"带有时区信息的datetime: {aware_dt}")
print(f"时区信息: {aware_dt.tzinfo}")
print(f"UTC偏移量: {aware_dt.utcoffset()}") # 上海是东八区，所以偏移量是+8小时

带有时区信息的datetime: 2025-10-11 14:00:00+08:00
时区信息: tzfile('PRC')
UTC偏移量: 8:00:00
